# Negative sampling

You may have noticed that word2vec is really slow to train. Especially with big (> 50 000) vocabularies. Negative sampling is the solution.

The task is to implement word2vec with negative sampling. In more detail: you should implement two ways of negative sampling.

## Vanilla negative sampling

This is what was discussed in Stanford lecture. The main idea is in the formula:

$$ L = \log\sigma(u^T_o u_c) + \sum^k_{i=1} \mathbb{E}_{j \sim P(w)}[\log\sigma(-u^T_j, u_c)]$$

Where $\sigma$ - sigmoid function, $u_c$ - central word vector, $u_o$ - context (outside of the window) word vector, $u_j$ - vector or word with index $j$.

The first term calculates the similarity between positive examples (word from one window)

The second term is responsible for negative samples. $k$ is a hyperparameter - the number of negatives to sample.
$\mathbb{E}_{j \sim P(w)}$
means that $j$ is distributed accordingly to unigram distribution, but it is better to use $P^{3/4}(w)$ (empirical results) and you can experiment with some other approaches (for example, try to use uniform distribution).

Thus, it is only required to calculate the similarity between positive samples and some other negatives. Not across all the vocabulary.

Useful links:
1. [Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/pdf/1301.3781.pdf)
1. [Distributed Representations of Words and Phrases and their Compositionality](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)

## Batch-transpose trick for negative sampling

But we can do better. Maybe we don't need to compute vectors for negative samples at all, because we already have a batch of training data and (hopefully) examples in the batch are highly decorrelated.

Let's assume we work with Skip-gram model.

Let $S$ be a batch of _L2-normalized_ word vectors `(batch_size, 2 * window_size + 1, word_vector_dim)`.

```python
x = 0.0
for batch_idx in range(batch):
    w = S[batch_idx, :, :]
    x += np.sum(w.T @ w - 1.)

y = 0.0
for window_idx in range(window):
    b = S[:, window_idx, :]
    y += np.sum(b.T @ b)

loss = -x + y```

Think about this loss and compare it to vanilla negative sampling.

Implement word2vec with batch-transpose trick. Modify the formula, if needed.

If you are interested: [more info](https://www.tensorflow.org/extras/candidate_sampling.pdf) on other methods of candidate sampling.

**Results of this task** are the very same as in task 3, **plus**:
 * implement two models (one with vanilla negative sampling and the other with batch-transpose trick)
 * compare all of the models from tasks 3-5. In terms of time and number of iterations until convergence and the quality of the resulting vectors.
 * answer the questions

### Questions:
1. Explain the batch-transpose trick formula in your own words. How would you name x, y, w and b?
1. Should it be modified to serve as a word2vec loss? If yes, how?
1. Is it possible to do the same trick with CBOW model? If yes, how?
1. Does it matter how the batch is made in the case of batch-transpose trick? In the case of vanilla negative sampling?




## Answers:
  1. w is 
  1. _
  1. _
  1. _

#### Imports

In [7]:
from skipgram import NegativeSamplingBatcher, NegativeSamplingSkipGram
import torch
import gc
import datetime
import pickle
import numpy as np
import pandas as pd

# for visualisation
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

#### Constants

In [8]:
# select whether to train model during this run or just load it from saved file
TRAIN = False

VOCAB_SIZE = 20000
BATCH_SIZE = 1024
EMBEDDINGS_DIM = 100
EPOCH_NUM = 2
WINDOW_SIZE = 3
N_NEGATIVE_SAMPLES = 15
LOGS_PERIOD = 50
MODEL_NAME = 'neg_sam'
np.random.seed(42)

#### Select device

In [9]:
USE_GPU = True

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

if device.__str__() == 'cpu':
    tensor_type = torch.LongTensor
else:
    tensor_type = torch.cuda.LongTensor

print('using device:', device)
print(f'using {tensor_type} type')

using device: cpu
using <class 'torch.LongTensor'> type


#### Load corpus into batcher

In [ ]:
text = []
with open('./data/text8', 'r') as text8:
    text = text8.read().split()

# text = ['first', 'used', 'against', 'early', 'working', 'radicals', 'including', 'class', 'other']
batcher = NegativeSamplingBatcher(text, vocab_size=VOCAB_SIZE, window_size=WINDOW_SIZE,
                                  batch_size=BATCH_SIZE, n_negative_examples=N_NEGATIVE_SAMPLES)
# free memory
text = []
gc.collect()

#### Create and train model

In [ ]:
if TRAIN:
    loss_history = []
    corpus_size = len(batcher._corpus_tokens)

    model = NegativeSamplingSkipGram(VOCAB_SIZE, EMBEDDINGS_DIM)
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

In [ ]:
if TRAIN:
    learning_started = datetime.datetime.now()

    cumulative_loss = 0
    for epoch in range(EPOCH_NUM):
        for i, (context, target, negatives) in enumerate(batcher):
            if len(context) < BATCH_SIZE:
                continue
            # Transform tokens from numpy to torch.Tensor
            tensor_context = torch.from_numpy(context).type(tensor_type)
            tensor_target = torch.from_numpy(target).type(tensor_type)
            tensor_negatives = torch.from_numpy(negatives).type(tensor_type)
            # Send tensors to the selected device
            tensor_context.to(device)
            tensor_target.to(device)
            tensor_negatives.to(device)
            
            model.zero_grad()
            loss = model(tensor_target, tensor_context, tensor_negatives)
            loss.backward()
            optimizer.step()
            cumulative_loss += loss

            if i % LOGS_PERIOD == 0:
                print(f'loss on {(i * BATCH_SIZE / corpus_size) * 100:.1f}%:' + \
                      f'{(cumulative_loss / LOGS_PERIOD) :.7f}')
                loss_history.append(loss.data.cpu().item())
                cumulative_loss = 0
        
        
        # after every epoch we save:
                                    # the model
                                    # loss history
        learning_ended = datetime.datetime.now()
        learning_time = (learning_ended - learning_started).total_seconds()
        learning_ended = learning_ended.strftime("%H-%M %d-%m-%Y")

        torch.save(model, f'./models/{MODEL_NAME}(finished-{learning_ended})(epochs_completed-{epoch})(epoch_num-{EPOCH_NUM})' + \
                   f'(vocab-{VOCAB_SIZE})(batch-{BATCH_SIZE})' + \
                   f'(emb-{EMBEDDINGS_DIM})(wind-{WINDOW_SIZE})(consumed-{learning_time})'+ \
                   f'.pytorchmodel')

        with open(f'./data/loss/loss_{MODEL_NAME}(finished-{learning_ended})(epochs_completed-{epoch})(epoch_num-{EPOCH_NUM})' + \
                  f'(vocab-{VOCAB_SIZE})(batch-{BATCH_SIZE})' + \
                  f'(emb-{EMBEDDINGS_DIM})(wind-{WINDOW_SIZE})(consumed-{learning_time})'+ \
                  f'.pickle', 'wb') as f:
            pickle.dump(loss_history, f)
        print('Model saved succesfully')
        

#### Plotting loss

In [ ]:
from utils import plot_moving_average

if not TRAIN:
    loss_file_location = './data/loss/neg_sam(finished-19-02 08-03-2019)(epochs_completed-1)(epoch_num-2)(vocab-20000)(batch-1024)(emb-100)(wind-3)(consumed-778.097815).pickle'
    with open(loss_file_location, 'rb') as f:
        loss_history = pickle.load(f)

In [ ]:
plot_moving_average(pd.Series(loss_history), 64, plot_actual=False)

#### Function to map token (and word) to corresponding word vector

In [ ]:
from utils import EmbeddingsEval
if not TRAIN:
    model_file_location ='./models/neg_sam(finished-19-02 08-03-2019)(epochs_completed-1)(epoch_num-2)(vocab-20000)(batch-1024)(emb-100)(wind-3)(consumed-778.097815).pytorchmodel'
    model = torch.load(model_file_location, map_location='cpu')
model

In [ ]:
intrinsic_matrix = model.get_intrinsic_matrix()

emb_eval = EmbeddingsEval(intrinsic_matrix, words_to_tokens=batcher.words_to_tokens,
                          tokens_to_words=batcher.tokens_to_words)

In [ ]:
emb_eval.tokens_to_embeddings([1, 2, 3])
emb_eval.words_to_embeddings(['integrity'])

#### Beautiful visualizations
Take top n most popular words

In [ ]:
num_words = 200
words = batcher.tokens_to_words(np.arange(0, num_words))
embeddings = emb_eval.tokens_to_embeddings(np.arange(0, num_words))

In [ ]:
pca = PCA(n_components=2)
points2d = pca.fit_transform(embeddings)


fig, ax = plt.subplots(figsize=(16, 16))
ax.scatter(points2d[:, 0], points2d[:, 1])

for i, word in enumerate(words):
    ax.annotate(word, (points2d[i, 0] + 0.01, points2d[i, 1] + 0.01), fontsize='small')

#### qualitative evaluations of word vectors: nearest neighbors, word analogies

In [ ]:
for token_list in emb_eval.words_to_neighbors(['war', 'france', 'drink']):
    print(batcher.tokens_to_words(token_list))

In [ ]:
similar = emb_eval.most_similar(positive=['france', 'berlin'], negative=['paris'])
similar